In [ ]:
import cv2
import numpy as np
import glob
from skin_detect import skinDetector
import matplotlib.pyplot as plt

In [ ]:
img_paths = glob.glob('images/*')

In [ ]:
def skinImageCrop(image_path):    
    # 肌探知アルゴリズムreq
    skindetector = skinDetector(image_path)
    skindetector.find_skin()
    
    # 4段階のBlurとThresholdをかけてノイズを除去する
    blurred = cv2.GaussianBlur(skindetector.binary_mask_image, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 254, 255, 0)
    blurred = cv2.blur(skindetector.binary_mask_image, (100, 100))
    _, thresh = cv2.threshold(blurred, 250, 255, 0)
    blurred = cv2.blur(skindetector.binary_mask_image, (50, 50))
    _, thresh = cv2.threshold(blurred, 250, 255, 0)
    blurred = cv2.blur(skindetector.binary_mask_image, (25, 25))
    _, thresh = cv2.threshold(blurred, 250, 255, 0)
    
    #　輪郭探知
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_area = [cv2.contourArea(con) for con in contours] # 輪郭内の面積
    target = contours[np.argmax(contours_area)] # 面積が最大のものが肌
    
    # 輪郭を多面体で近似してバウンディングボックスを取得する
    polygon = cv2.approxPolyDP(target, 3, True)
    x,y,w,h = cv2.boundingRect(polygon)
    centers, radius = cv2.minEnclosingCircle(polygon)
    
    # 多面体に外接しているバウンディングボックスを内接するように縮める
    shrink = np.rint(radius*(1 - np.cos(np.pi/4)))
    offset = 10 # 微調整
    x = round(x + shrink + offset)
    y = round(y + shrink + offset)
    w = round(w - 2*(shrink + offset))
    h = round(h - 2*(shrink + offset))
    
    #　クロップ
    return skindetector.image[y:y+w,x:x+h,:]

In [ ]:
for idx, img in enumerate(img_paths):
    try:
        cropped = skinImageCrop(img)
        cv2.imwrite('image/' + str(idx) + '_cropped.jpg', cropped)
    except ValueError:
        continue